In [46]:
import requests
import pandas as pd
#import streamlit as st


# @st.cache_data
def fetch_data_from_api(url: str, offset: int, limit: int) -> dict:
    params = {'limit': limit, 'offset': offset}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        raise Exception(f"Failed API request: Status {response.status_code}")
    

# @st.cache_data
def fetch_all_data(url: str) -> pd.DataFrame:
    all_data = []
    limit = 100
    offset = 0
    total_records = None

    while total_records is None or offset < total_records:
        data = fetch_data_from_api(url, offset, limit)
        all_data.extend(data['results'])

        if total_records is None:
            total_records = data['total_count']
        offset += limit

    return pd.json_normalize(all_data)

# Utilisez la fonction fetch_all_data et gérez les exceptions ou les notifications en dehors de la fonction
try:
    url_api = "https://data.regionreunion.com/api/explore/v2.1/catalog/datasets/population-francaise-communespublic/records"
    populations = fetch_all_data(url_api)
except Exception as e:
    print(f"Erreur lors de la récupération des données : {e}")


try:
    url_api = "https://opendata-reunion.edf.fr/api/explore/v2.1/catalog/datasets/consommation-annuelle-par-commune0/records"
    conso_nrj = fetch_all_data(url_api)
except Exception as e:
    print(f"Erreur lors de la récupération des données : {e}")

df_population = (populations.groupby("annee_utilisation", as_index=False)["population_totale"]
      .sum()
     )
#renomme annee_utilisation en année pour la fusion
df_population = df_population.rename(columns={"annee_utilisation": 'annee'})
codes1 = sorted(df_population["annee"].unique())
df_conso = (conso_nrj.groupby("annee", as_index=False)["consommation_mwh"]
      .sum()
     )
codes2 = sorted(df_conso["annee"].unique())
df_conso["annee"] = df_conso["annee"].astype(str)

# Fusionner les DataFrames sur la colonne 'annee'
population = pd.merge(df_population, df_conso, on='annee', how='inner')

#chargement du fichier geoson
import json

with open("./communes-974-la-reunion.geojson") as f:
    communes = json.loads(f.read())
codes3 = sorted([f["properties"]["code"] for f in communes["features"]])
#codes3 = [code + '974' for code in codes3]

#creation dataframe populations avec clé_unique code_commune
df_population_codcom = (populations.groupby("code_insee_commune", as_index=False)["population_totale"]
      .sum()
     )

#codes4 = sorted(df_population_codcom["code_insee_commune"].unique())
# Créer la clé unique en concaténant 'code_commune' et 'annee_utilisation'
df_population_codcom['cle_unique'] = populations['code_insee_commune'].astype(str) + '_' + populations['annee_utilisation'].astype(str)
codes6 = sorted(df_population_codcom["cle_unique"].unique())

#creation dataframe conso_nrj avec clé_unique code_insee
df_conso_edf_codcom = (conso_nrj.groupby("code_insee", as_index=False)["consommation_mwh"]
      .sum()
     )
df_conso_edf_codcom['cle_unique'] = conso_nrj['code_insee'].astype(str) + '_' + conso_nrj['annee'].astype(str)
codes6 = sorted(df_conso_edf_codcom["cle_unique"].unique())

#Fusionner les 2 nouveau dataframe
df_merged = pd.merge(df_population_codcom, df_conso_edf_codcom)
df_merged


,code_insee_commune,population_totale,cle_unique,code_insee,consommation_mwh


In [13]:
# Grouper par 'code_insee' et calculer la somme de 'population_totale' et 'autre_colonne'
df_merged['annee_fusionnee'] = df_population['annee'].astype(str) + '-' + df_conso['annee'].astype(str)

# Obtenir les années uniques à partir de df_population_codcom
annees_uniques = df_population['annee'].unique()

# Obtenir la plage d'années à partir de df_conso_edf_codcom
annee_min = df_population['annee'].min()
annee_max = df_population['annee'].max()

# Filtrer les années uniques dans la plage d'années spécifiée
annees_uniques_filtrees = annees_uniques[(annees_uniques >= annee_min) & (annees_uniques <= annee_max)]

# Créer un DataFrame avec les années filtrées
df_resultat = pd.DataFrame({'annee': annees_uniques_filtrees})

# Afficher le résultat
print(df_resultat)

   annee
0   2009
1   2010
2   2011
3   2012
4   2013
5   2014
6   2015
7   2016
8   2017
9   2018
10  2019


In [19]:
df_population_codcom

,code_insee_commune,population_totale
0,97401,119538
1,97402,133642
2,97403,70281
3,97404,152391
4,97405,130205
5,97406,60566
6,97407,411129
7,97408,337904
8,97409,601128
9,97410,394936


In [18]:
# Créer une table relationnelle en fusionnant les deux DataFrames sur la colonne "code_insee"
table_relationnelle = pd.merge(df_population_codcom, df_resultat, left_on='annee', right_on='annee', how='inner')

KeyError: 'annee'